Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# 3D regression example based on DenseNet

This tutorial shows an example of 3D regression task based on DenseNet and array format transforms.

Here, the task is given to predict the ages of subjects from MR imagee.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/3d_regression/densenet_training_array.ipynb)

## Setup environment

In [28]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"

## Setup imports

In [1]:
import logging
import os
import sys
import shutil
import tempfile

import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)
from monai.networks.nets import Regressor

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

MONAI version: 1.5.dev2445
Numpy version: 1.26.2
Pytorch version: 2.5.1+cpu
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 2af9926d853086b264680adcf954bf3232f5ec32
MONAI __file__: c:\Users\<username>\AppData\Local\Programs\Python\Python311\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.11.4
Pillow version: 10.1.0
Tensorboard version: 2.18.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.66.4
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 6.0.0
pandas version: 2.1.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED

## Setup data directory

In [2]:
# Set data directory
directory = r'C:\Users\Hikari20220126i712th\Downloads\IXI-T1\MONAI'
os.makedirs(directory, exist_ok=True)
root_dir = directory

print(root_dir)

C:\Users\Hikari20220126i712th\Downloads\IXI-T1\MONAI


In [3]:
import os
import pandas as pd
import numpy as np
from glob import glob

# 讀取 Excel 檔案
excel_file = r'C:\Users\Hikari20220126i712th\Downloads\IXI-T1\MONAI\MONAI_trainval.xlsx'  # 替換為您的 Excel 檔案路徑
df = pd.read_excel(excel_file)

# 根目錄與子資料夾
root_dir = r'C:\Users\Hikari20220126i712th\Downloads\IXI-T1\MONAI'  # 替換為 NIfTI 檔案的根目錄
nii_folder = r'C:\Users\Hikari20220126i712th\Downloads\IXI-T1\MONAI\ixi'  # 假設 NIfTI 檔案存放於 'ixi' 子資料夾
nii_dir = os.path.join(root_dir, nii_folder)

# 初始化影像路徑和年齡清單
images = []
ages = []

# 從 Excel 文件中動態匹配影像檔案
for _, row in df.iterrows():
    file_id = row["IXI_ID"]  # 從 Excel 中獲取 ID（如 "IXI314"）
    age = row["AGE"]  # 從 Excel 中獲取對應年齡

    # 根據 ID 搜索對應的 NIfTI 檔案（支援模糊匹配）
    search_pattern = os.path.join(nii_dir, f"{file_id}*.nii.gz")
    matching_files = glob(search_pattern)

    # 如果找到對應檔案，加入清單
    if len(matching_files) > 0:
        images.append(matching_files[0])  # 添加匹配到的檔案路徑
        ages.append(age)  # 添加對應的年齡
    else:
        print(f"File not found for ID: {file_id}")

# 將年齡轉為 NumPy 陣列，與原始程式保持一致
ages = np.array(ages)

# 確認資料載入成功
print(f"Loaded {len(images)} images and {len(ages)} age labels.")


Loaded 446 images and 446 age labels.


## Create data loaders

In [4]:
####改訓練集 驗證集比例
from sklearn.model_selection import train_test_split

# 定義資料轉換
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 96)), RandRotate90()])
val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 96))])

# 使用 train_test_split 分割資料，70% 為訓練集，30% 為驗證集
train_images, val_images, train_ages, val_ages = train_test_split(
    images, ages, test_size=0.3, random_state=42  # 驗證集比例為 30%
)

# 檢查資料分割結果
print(f"Training samples: {len(train_images)}, Validation samples: {len(val_images)}")

# 定義訓練資料集與 DataLoader
train_ds = ImageDataset(image_files=train_images, labels=train_ages, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2, pin_memory=pin_memory)

# 定義驗證資料集與 DataLoader
val_ds = ImageDataset(image_files=val_images, labels=val_ages, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=2, shuffle=False, num_workers=2, pin_memory=pin_memory)

# 測試 DataLoader 的第一個批次
check_ds = ImageDataset(image_files=images, labels=ages, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=3, num_workers=2, pin_memory=pin_memory)

im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label, label.shape)


Training samples: 312, Validation samples: 134
<class 'monai.data.meta_tensor.MetaTensor'> torch.Size([3, 1, 96, 96, 96]) tensor([38.7800, 46.7100, 34.2400], dtype=torch.float64) torch.Size([3])


## Create model and train

In [5]:
model = Regressor(in_shape=[1, 96, 96, 96], out_shape=1, channels=(16, 32, 64, 128, 256), strides=(2, 2, 2, 2))
if torch.cuda.is_available():
    model.cuda()
# It is important that we use nn.MSELoss for regression.
loss_function = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()
max_epochs = 100 #修改epochs數

lowest_rmse = sys.float_info.max
for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0

    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        all_labels = []
        all_val_outputs = []
        for val_data in val_loader:
            val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
            all_labels.extend(val_labels.cpu().detach().numpy())
            with torch.no_grad():
                val_outputs = model(val_images)
                flattened_val_outputs = [val for sublist in val_outputs.cpu().detach().numpy() for val in sublist]
                all_val_outputs.extend(flattened_val_outputs)

        mse = np.square(np.subtract(all_labels, all_val_outputs)).mean()
        rmse = np.sqrt(mse)

        if rmse < lowest_rmse:
            lowest_rmse = rmse
            lowest_rmse_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_array.pth")
            print("saved new best metric model")

        print(f"Current epoch: {epoch+1} current RMSE: {rmse:.4f} ")
        print(f"Best RMSE: {lowest_rmse:.4f} at epoch {lowest_rmse_epoch}")
        writer.add_scalar("val_rmse", rmse, epoch + 1)

print(f"Training completed, lowest_rmse: {lowest_rmse:.4f} at epoch: {lowest_rmse_epoch}")
writer.close()

----------
epoch 1/100


c:\Users\Hikari20220126i712th\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_tensor.py:1512: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  ret = func(*args, **kwargs)


1/156, train_loss: 1703.8031
2/156, train_loss: 2361.9702
3/156, train_loss: 1484.5273
4/156, train_loss: 1560.8220
5/156, train_loss: 1342.9387
6/156, train_loss: 630.7622
7/156, train_loss: 1412.1378
8/156, train_loss: 1298.9618
9/156, train_loss: 2543.3772
10/156, train_loss: 904.5133
11/156, train_loss: 64.3124
12/156, train_loss: 424.7197
13/156, train_loss: 1130.8608
14/156, train_loss: 400.0118
15/156, train_loss: 76.0546
16/156, train_loss: 611.3971
17/156, train_loss: 289.7202
18/156, train_loss: 282.7812
19/156, train_loss: 582.6309
20/156, train_loss: 479.6921
21/156, train_loss: 772.2903
22/156, train_loss: 31.1914
23/156, train_loss: 649.1265
24/156, train_loss: 87.7614
25/156, train_loss: 158.9233
26/156, train_loss: 289.8881
27/156, train_loss: 529.4741
28/156, train_loss: 133.6092
29/156, train_loss: 129.5031
30/156, train_loss: 437.5203
31/156, train_loss: 235.1494
32/156, train_loss: 203.9000
33/156, train_loss: 390.6538
34/156, train_loss: 208.0409
35/156, train_loss

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr" #"all"


## Cleanup data directory

Remove directory if a temporary was used.

In [7]:
if directory is None:
    shutil.rmtree(root_dir)